In [3]:
import tweepy
import requests
from PIL import Image
import io
import torch
from torchvision.transforms import ToTensor
from ipynb.fs.full.cnn import FashionCNN  # Import the CNN model

Epoch 1/10, Loss: 0.5294
Epoch 2/10, Loss: 0.3399
Epoch 3/10, Loss: 0.2944
Epoch 4/10, Loss: 0.2628
Epoch 5/10, Loss: 0.2366
Epoch 6/10, Loss: 0.2164
Epoch 7/10, Loss: 0.1993
Epoch 8/10, Loss: 0.1799
Epoch 9/10, Loss: 0.1639
Epoch 10/10, Loss: 0.1484


In [4]:
# https://developer.twitter.com/en/portal/projects-and-apps
# Authenticate and set up access to the Twitter API
consumer_key = "x"
consumer_secret = "x"
access_token = "x-x"
access_token_secret = "x"

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [6]:
# Load the saved CNN model
model = FashionCNN()
model.load_state_dict(torch.load("fashion_cnn.pth"))
model.eval()

FashionCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
# WILL NEED TOKENS AND PROPER AUTHORIZATION FOR THIS TO WORK

# Twitter @s to search from
accounts = ["Zara", "sheinofficial", "hm"]
hashtags = []

# Initialize the dictionary to store clothing types and frequencies
trending_clothing = {}

# Retrieve and classify images from Twitter
for account in accounts:
    tweets = api.user_timeline(screen_name=account, count=10, include_rts=False, tweet_mode="extended")
    for tweet in tweets:
        if 'media' in tweet.entities:
            for media in tweet.entities['media']:
                if media['type'] == 'photo':
                    image_url = media['media_url']
                    response = requests.get(image_url)
                    image = Image.open(io.BytesIO(response.content)).convert("RGB")
                    image_tensor = ToTensor()(image).unsqueeze(0)

                    # Classify the clothing type using the CNN model
                    output = model(image_tensor)
                    _, predicted = torch.max(output.data, 1)
                    predicted_class = predicted.item()

                    # Update the clothing type frequencies
                    trending_clothing[predicted_class] = trending_clothing.get(predicted_class, 0) + 1

                    print(f"Tweet ID: {tweet.id}, Account: {account}, Predicted Class: {predicted_class}")

In [7]:
# Visualizations in visualizations.ipynb